In [ ]:
# Import all dependencies

from __future__ import print_function, absolute_import
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.metrics import matthews_corrcoef, balanced_accuracy_score
import multiprocessing
import scipy.stats
import sys

In [ ]:
# Load the dataset
import src.data_loader as dl
subIDs, data, labels = dl.load_processed_data_N_subjects_allchans('data/', Nsub=14)

### Obtain a STFT for a given epoch

In [4]:
def get_stft(data_array):
    f, t, data_stft = np.abs(signal.stft(data_array, fs=100, nperseg=64, noverlap=48, axis=-1))
    data_stft = np.moveaxis(data_stft, 1, -1)
    return data_stft

### Normalize the STFT to the subject's overall EEG spectrum (mean, std)

In [5]:
def get_stft_zscore(stft):
    
    # move trials to the last axis
    data_stft_mean = np.mean(data_stft, axis=(2,3))
    data_stft_mean = np.repeat(data_stft_mean[:,:,np.newaxis], data_stft.shape[2], axis=2)
    data_stft_mean = np.repeat(data_stft_mean[:,:,:, np.newaxis], data_stft.shape[3], axis=3)

    data_stft_std = np.std(data_stft, axis=(2,3))
    data_stft_std = np.repeat(data_stft_std[:,:,np.newaxis], data_stft.shape[2], axis=2)
    data_stft_std = np.repeat(data_stft_std[:,:,:, np.newaxis], data_stft.shape[3], axis=3)
    
    stft_norm = (data_stft - data_stft_mean) / data_stft_std
    
    return stft_norm

### Split the data into train and test based on subjects

In [98]:
xtrain, xtest, ytrain, ytest = train_test_split(data, labels, test_size=0.25, random_state=42)

In [100]:
def generate_features(stft_z, labels_):
      
    # Then get the frames as feature vectors
    data_array = label_array = np.array([])
    for trial in np.arange(stft_z.shape[-1]):
        
        for timepoint in np.arange(stft_z.shape[-2]):
            data_array = np.append(data_array, stft_z[:, :, timepoint, trial].ravel(), axis=0)
            l_t = np.where(labels_[trial,:]==1)
            label_array = np.append(label_array, l_t)
    
    X = np.reshape(data_array, (-1, stft_z.shape[0]*stft_z.shape[1]))
    y = label_array.astype(int)
    y[y==6] = 0
    y[y>0] = 1

    return X, y

In [101]:
for sub in range(len(xtrain)):
    print("Subject {} out of {}".format(sub+1, len(xtrain)))
    data_stft = get_stft(xtrain[sub])
    data_stft_norm = get_stft_zscore(data_stft)

    if sub==0:
        X, y = generate_features(data_stft_norm, ytrain[sub])
    else:
        X_sub, y_sub = generate_features(data_stft_norm, ytrain[sub])
        X = np.append(X, X_sub, axis=0)
        y = np.append(y, y_sub, axis=0)

Subject 1 out of 10
Subject 2 out of 10
Subject 3 out of 10
Subject 4 out of 10
Subject 5 out of 10
Subject 6 out of 10
Subject 7 out of 10
Subject 8 out of 10
Subject 9 out of 10
Subject 10 out of 10


In [106]:
# Randomize the training set

ds = np.empty((X.shape[0], X.shape[1]+1))
ds[:,:-1]=X
ds[:,-1] = y
np.random.seed=42
np.random.shuffle(ds)

In [107]:
clf = SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=True, random_state=42, shrinking=True, tol=0.001,
  verbose=0)

clf.fit(ds[:,:-1],ds[:,-1])

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=42, shrinking=True, tol=0.001,
    verbose=0)

In [108]:
def generate_test_features(stft_epoch):
    data_array = label_array = np.array([])        
    for timepoint in np.arange(stft_epoch.shape[-1]):
        data_array = np.append(data_array, stft_epoch[:, :, timepoint].ravel(), axis=0)
    X = np.reshape(data_array, (-1, stft_epoch.shape[0]*stft_epoch.shape[1]))
    return X

In [109]:
scores = np.empty((len(xtest),))
for sub in range(len(xtest)):
    print("Subject {} out of {}".format(sub+1, len(xtest)))
    data_stft = get_stft(xtest[sub])
    data_stft_norm = get_stft_zscore(data_stft)
    y_m = np.array([])
    for epoch in range(data_stft_norm.shape[-1]):
        tpc = np.floor(data_stft_norm.shape[-1]/100)
        if epoch%10==0:
            print("Epoch {} out of {}".format(epoch+1, data_stft_norm.shape[-1]))
        X = generate_test_features(data_stft_norm[:,:,:,epoch])
        y_pred = clf.predict(X)
        y_m = np.append(y_m, np.median(y_pred))
    y = ytest[sub]
    label_array = np.array([])
    for i in range(y.shape[0]):
        l_t = np.where(y[i,:]==1)
        label_array = np.append(label_array, l_t)
            
    y_test = label_array.astype(int)
    y_test[y_test==6] = 0
    y_test[y_test>0] = 1
    
    score = balanced_accuracy_score(y_m.astype(int), y_test.astype(int))
    print("\n\nScore is %0.3f \n\n" %(score))
    scores[sub] = score

Subject 1 out of 4
Epoch 1 out of 100
Epoch 11 out of 100
Epoch 21 out of 100
Epoch 31 out of 100
Epoch 41 out of 100
Epoch 51 out of 100
Epoch 61 out of 100
Epoch 71 out of 100
Epoch 81 out of 100
Epoch 91 out of 100


Score is 1.000 


Subject 2 out of 4
Epoch 1 out of 300
Epoch 11 out of 300
Epoch 21 out of 300
Epoch 31 out of 300
Epoch 41 out of 300
Epoch 51 out of 300
Epoch 61 out of 300
Epoch 71 out of 300
Epoch 81 out of 300
Epoch 91 out of 300
Epoch 101 out of 300
Epoch 111 out of 300
Epoch 121 out of 300
Epoch 131 out of 300
Epoch 141 out of 300
Epoch 151 out of 300
Epoch 161 out of 300
Epoch 171 out of 300
Epoch 181 out of 300
Epoch 191 out of 300
Epoch 201 out of 300
Epoch 211 out of 300
Epoch 221 out of 300
Epoch 231 out of 300
Epoch 241 out of 300
Epoch 251 out of 300
Epoch 261 out of 300
Epoch 271 out of 300
Epoch 281 out of 300
Epoch 291 out of 300


Score is 1.000 


Subject 3 out of 4
Epoch 1 out of 300
Epoch 11 out of 300
Epoch 21 out of 300
Epoch 31 out of 300
Epoch 